In [2]:
import pandas as pd
from io import StringIO
pd.set_option('display.max_columns', None)

In [3]:
#Import dataframe
df_100pos = pd.read_csv('DataFiles/merged_df_100pos.csv', low_memory=False)

In [34]:
# Adding additional advanced metric columns

# Shooting efficiency 
df_100pos['FG3A%'] = df_100pos['FG3A'] / df_100pos['FGA']
df_100pos['PTS/FGA'] = df_100pos['PTS'] / df_100pos['FGA']
df_100pos['FG3M/FGM'] = df_100pos['FG3M'] / df_100pos['FGM']
df_100pos['FTA/FGA'] = df_100pos['FTA'] / df_100pos['FGA']

# True Shooting Percentage
df_100pos['TRU%'] = 0.5 * df_100pos['PTS'] / (df_100pos['FGA'] + 0.475 * df_100pos['FTA'])

# Assists to Turnover Ratio
df_100pos['AST_TOV'] = df_100pos['AST'] / df_100pos['TOV']

df_100pos['POSSESSION'] = df_100pos.apply(lambda row: row['FGA'] + 0.44 * row['FTA'] - row['OREB'] + row['TOV'], axis=1)

df_100pos[(df_100pos['PLAYER_ID'] == 1629029)]

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,Year,Season,COLLEGE,COUNTRY,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,JERSEY_NUMBER,POSITION,HEIGHT,WEIGHT,FROM_YEAR,TO_YEAR,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%,AST_TOV,POSSESSION
17024,1629029,Luka Doncic,1610612742,DAL,20.0,72,28,44,0.389,47.4,10.3,24.2,0.427,3.4,10.5,0.327,7.1,9.9,0.713,1.8,9.8,11.5,8.8,5.1,1.6,0.5,1.1,2.8,8.3,31.2,2018-19,Regular Season,NaN,Slovenia,2018,1,3,77,F,6-7,230.0,2018,2023,0.433884,1.289256,0.330097,0.409091,0.539746,1.725490,31.856
17025,1629029,Luka Doncic,1610612742,DAL,21.0,61,36,25,0.590,47.3,13.4,29.0,0.463,4.0,12.5,0.316,9.8,13.0,0.758,1.8,11.4,13.2,12.4,6.0,1.4,0.3,1.5,3.5,10.5,40.6,2019-20,Regular Season,NaN,Slovenia,2018,1,3,77,F,6-7,230.0,2018,2023,0.431034,1.400000,0.298507,0.448276,0.577114,2.066667,38.920
17026,1629029,Luka Doncic,1610612742,DAL,21.0,6,2,4,0.333,46.7,13.9,27.9,0.500,3.5,9.6,0.364,9.2,13.9,0.656,0.9,12.0,12.9,11.3,6.8,1.5,0.7,1.1,3.7,12.0,40.5,2019-20,Playoffs,NaN,Slovenia,2018,1,3,77,F,6-7,230.0,2018,2023,0.344086,1.451613,0.251799,0.498208,0.586914,1.661765,39.916
17027,1629029,Luka Doncic,1610612742,DAL,22.0,66,40,26,0.606,48.4,13.8,28.9,0.479,4.1,11.7,0.350,7.4,10.1,0.730,1.2,10.1,11.3,12.1,6.0,1.4,0.8,1.1,3.3,7.8,39.1,2020-21,Regular Season,NaN,Slovenia,2018,1,3,77,F,6-7,230.0,2018,2023,0.404844,1.352941,0.297101,0.349481,0.580162,2.016667,38.144
17028,1629029,Luka Doncic,1610612742,DAL,22.0,7,3,4,0.429,50.8,17.4,35.4,0.490,5.6,13.7,0.408,4.9,9.2,0.529,0.7,9.2,9.9,13.0,5.8,1.6,0.5,0.9,3.1,8.1,45.2,2020-21,Playoffs,NaN,Slovenia,2018,1,3,77,F,6-7,230.0,2018,2023,0.387006,1.276836,0.321839,0.259887,0.568268,2.241379,44.548
17029,1629029,Luka Doncic,1610612742,DAL,23.0,65,44,21,0.677,50.0,13.9,30.5,0.457,4.4,12.4,0.353,7.9,10.6,0.744,1.2,11.7,12.9,12.3,6.3,1.6,0.8,1.0,3.1,8.7,40.1,2021-22,Regular Season,NaN,Slovenia,2018,1,3,77,F,6-7,230.0,2018,2023,0.406557,1.314754,0.316547,0.347541,0.564232,1.952381,40.264
17030,1629029,Luka Doncic,1610612742,DAL,23.0,15,7,8,0.467,51.4,14.9,32.8,0.455,4.8,13.8,0.345,9.7,12.6,0.770,0.9,12.8,13.7,8.9,5.5,2.5,0.8,1.5,4.1,10.6,44.3,2021-22,Playoffs,NaN,Slovenia,2018,1,3,77,F,6-7,230.0,2018,2023,0.420732,1.350610,0.322148,0.384146,0.571097,1.618182,42.944
17031,1629029,Luka Doncic,1610612742,DAL,24.0,66,33,33,0.500,49.0,14.8,29.7,0.496,3.8,11.1,0.342,10.6,14.2,0.742,1.1,10.6,11.7,10.9,4.8,1.8,0.7,0.9,3.4,9.9,43.9,2022-23,Regular Season,NaN,Slovenia,2018,1,3,77,F,6-7,230.0,2018,2023,0.373737,1.478114,0.256757,0.478114,0.602277,2.270833,39.648


In [35]:
# Replacing intermediary positions with one. All in all there are 3 positions - C, F and G. 
df_100pos['POSITION'] = df_100pos['POSITION'].str.replace('F-C', 'F').replace('C-F', 'C').replace('F-G', 'F').replace('G-F', 'G')

In [36]:
# European countries in basketball sense
europe = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark',
  'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland',
  'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands',
  'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 
  'United Kingdom', 'Norway', 'Switzerland', 'Belarus', 'Russia', 'Ukraine',
  'Moldova', 'Georgia', 'Armenia', 'Azerbaijan', 'Turkey', 'Israel', 'Albania',
  'Macedonia', 'Serbia', 'Montenegro', 'Bosnia and Herzegovina', 'Iceland']

In [37]:
# Define function for European countries
def filter_europe(df, column_name, europe):
    return df[df[column_name].isin(europe)]

In [38]:
# New object for European players playing at F in regular season games:
europe_F_regular = filter_europe(df_100pos, 'COUNTRY', europe)
europe_F_regular = europe_F_regular.loc[europe_F_regular['POSITION'].isin(['F']) & europe_F_regular['Season'].isin(['Regular Season'])]

In [39]:
# Define function for non-european countries
def filter_non_europe(df, column_name='COUNTRY', country_list=europe):
    return df[~df[column_name].isin(europe)]

In [40]:
# New object for Non-europe players playing at F in regular season games:
non_europe_F_regular = filter_non_europe(df_100pos)
non_europe_F_regular = non_europe_F_regular.loc[non_europe_F_regular['POSITION'].isin(['F']) & non_europe_F_regular['Season'].isin(['Regular Season'])]

In [41]:
# Number of positions per unique European player:
europe_F_regular.groupby('POSITION')['PLAYER_ID'].nunique()

POSITION
F    103
Name: PLAYER_ID, dtype: int64

In [42]:
# Number of positions per unique US player:
non_europe_F_regular.groupby('POSITION')['PLAYER_ID'].nunique()

POSITION
F    876
Name: PLAYER_ID, dtype: int64

In [45]:
non_europe_F_regular['HEIGHT'].describe()

count     4581
unique      10
top        6-9
freq      1104
Name: HEIGHT, dtype: object

In [43]:
import altair as alt
import pandas as pd
import altair_saver


# Annahme: Sie haben zwei DataFrames: data_per_min_europe_regular und data_per_min_usa_regular

# Fügen Sie eine neue Spalte hinzu, um anzuzeigen, ob der Spieler Europäer ist oder nicht
europe_F_regular['Europe'] = 'European'
non_europe_F_regular['Europe'] = 'Non-European'

# Kombinieren Sie die beiden DataFrames
combined_df = pd.concat([europe_F_regular, non_europe_F_regular])

# Liste der Metriken, die geplottet werden sollen
metrics = ['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 
           'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 
           'PF', 'PTS', 'FG3A%', 'PTS/FGA', 'FG3M/FGM', 'FTA/FGA', 
           'TRU%', 'AST_TOV', 'POSSESSION']

# Liste zum Speichern der Diagramme
charts = []

# Einen Bruchteil des kombinierten DataFrame auswählen
sampled_df = combined_df.sample(frac=0.5)  # Passen Sie den Bruchteil bei Bedarf an

# Für jede Metrik ein gruppiertes Balkendiagramm erstellen und die Diagramme in eine Liste speichern
for metric in metrics:
    chart = alt.Chart(sampled_df).mark_bar().encode(
        x=alt.X('Europe:N', title='Europe'),
        y=alt.Y(f'mean({metric}):Q', title=f'Average {metric}'),
        color=alt.Color('Europe:N', legend=alt.Legend(title='Europe'))
    ).properties(
        title=f'Average {metric} by Europe',
        width=400,
        height=300
    )
    charts.append(chart)

# Diagramme in sechs Zeilen stapeln
alt_row = alt.vconcat(*[alt.hconcat(*charts[i:i+4]) for i in range(0, len(charts), 4)])

# Die Diagramme anzeigen
alt_row


/opt/homebrew/Caskroom/miniconda/base/envs/nf_sql/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/homebrew/Caskroom/miniconda/base/envs/nf_sql/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/homebrew/Caskroom/miniconda/base/envs/nf_sql/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].a

alt.VConcatChart(...)